# Technical Indicators

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np

### Exponential Moving Average(EMA)
#### DF=Open, High, Low, Adj Close, Volume pandas dataframe,  period = window for exponential moving average

In [ ]:
def EMA(DF, period, min_period=1):
    fdf = pd.DataFrame()
    fdf["EMA"]= DF["Adj Close"].ewm(span=period, min_periods=min_period).mean()
    fdf.dropna(axis=0, inplace=True)
    return fdf

### Moving Average Convergence Divergence(MACD)  
####  DF=Open, High, Low, Adj Close, Volume pandas dataframe,  sp=slow moving average window, fp= Fast moving average window,  p= Signal window, this function returns a dataframe with MACD and Signal line 

In [2]:
def MACD(DF, sp, fp, p):
    df = DF.copy()
    df["slow_MA"]= df["Adj Close"].ewm(span=sp, min_periods=sp).mean()
    df["fast_MA"]= df["Adj Close"].ewm(span=fp, min_periods=fp).mean()
    fdf = pd.DataFrame()
    fdf['MACD']= df["fast_MA"]-df["slow_MA"]
    fdf['signal']= fdf['MACD'].ewm(span=p, min_periods=p).mean()
    fdf.dropna(axis=0, inplace=True)
    return fdf

### Average True Range(ATR)
####  DF=Open, High, Low, Adj Close, Volume pandas dataframe, n=window for ATR

In [ ]:
def ATR(DF, n):
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Adj Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Adj Close'].shift(1))
    df['TR']= df[['H-L', 'H-PC', 'L-PC']].max(axis=1, skipna=False)
    fdf= fdf = pd.DataFrame()
    fdf['Adj Close']=df['Adj Close']
    fdf['ATR']=df['TR'].rolling(n).mean()
    return fdf

### Bollinger's band
#### DF=Open, High, Low, Adj Close, Volume pandas dataframe , n=window for BB

In [ ]:
def BollBand(DF, n):
    df = DF.copy()
    fdf= pd.DataFrame()
    fdf['Adj Close']=df['Adj Close']
    fdf['MA']=df['Adj Close'].rolling(n).mean()
    fdf['BB_up']= fdf['MA'] +2*(fdf['MA'].rolling(n).std())
    fdf['BB_dn']= fdf['MA'] -2*(fdf['MA'].rolling(n).std())
    fdf['BB_wid']= fdf['BB_up']- fdf['BB_dn']
    fdf.dropna(inplace=True)
    return fdf

### Relative Strength Index(RSI) 
####  DF=Open, High, Low, Adj Close, Volume pandas dataframe, n=window for RSI

In [ ]:
def RSI(DF, n):
    df= DF.copy()
    df['delta']=df['Adj Close']- df['Adj Close'].shift(1)
    df['gain']= np.where(df['delta']>=0, df['delta'],0)
    df['loss']= np.where(df['delta']<0, abs(df['delta']),0)
    avg_gain =[]
    avg_loss =[]
    for i in range(len(df)):
        if i<n:
            avg_gain.append(np.NaN)
            avg_loss.append(np.NaN)
        elif i==n:
            avg_gain.append(df['gain'].rolling(n).mean()[n])
            avg_loss.append(df['loss'].rolling(n).mean()[n])
        elif i>n:
            avg_gain.append(((n-1)*avg_gain[i-1]+df['gain'][i])/n)
            avg_loss.append(((n-1)*avg_loss[i-1]+df['loss'][i])/n)
    df['avg_gain']=np.array(avg_gain)
    df['avg_loss']=np.array(avg_loss)
    df['RS']=  df['avg_gain']/df['avg_loss']
    fdf=pd.DataFrame()
    fdf['RSI']= 100-(100/(1+df['RS']))
    return fdf 

### Average Directional Movement Index(ADX)
####  DF=Open, High, Low, Adj Close, Volume pandas dataframe, n=window for ADX

In [ ]:
def ADX(DF, n):
    df = DF.copy()
    
    #True Range(TR) = max(H-L, H-PC, L-PC) 
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Adj Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Adj Close'].shift(1))
    df['TR']= df[['H-L', 'H-PC', 'L-PC']].max(axis=1, skipna=False)
    
    #If H-PH > PL-L then DMplus = H-PH else Zero
    #If PL-L > H-PH then DMminus = PL-L else Zero
    df['DMplus']= np.where((df['High']-df['High'].shift(1))>(df['Low'].shift(1)-df['Low']), (df['High']-df['High'].shift(1)), 0)
    df['DMplus']= np.where(df['DMplus']<0, 0, df['DMplus'])
    df['DMminus']= np.where((df['Low'].shift(1)-df['Low'])>(df['High']-df['High'].shift(1)), (df['Low'].shift(1)-df['Low']), 0)
    df['DMminus']= np.where(df['DMminus']<0, 0, df['DMminus'])
    
    #TR_avg = Smoothavg(TR), Similarly DMplus_avg and DMminus_avg
    TR_avg = []
    DMplus_avg = []
    DMminus_avg = []
    ADXf = []
    for i in range(len(df)):
        if i<n:
            TR_avg.append(np.NaN)
            DMplus_avg.append(np.NaN)
            DMminus_avg.append(np.NaN)
        elif i==n:
            TR_avg.append( df['TR'].rolling(n).sum()[n])
            DMplus_avg.append( df['DMplus'].rolling(n).sum()[n])
            DMminus_avg.append( df['DMminus'].rolling(n).sum()[n])
        elif i>n:
            TR_avg.append(TR_avg[i-1]-(TR_avg[i-1]/n)+df['TR'][i])
            DMplus_avg.append(DMplus_avg[i-1]-(DMplus_avg[i-1]/n)+df['DMplus'][i])
            DMminus_avg.append(DMminus_avg[i-1]-(DMminus_avg[i-1]/n)+df['DMminus'][i])
    df['TR_avg']=np.array(TR_avg)
    df['DMplus_avg']=np.array(DMplus_avg)
    df['DMminus_avg']=np.array(DMminus_avg)
    
    #DIplus = 100*(DMplus_avg/TR_avg)
    #DIminus = 100*(DMminus_avg/TR_avg)
    df['DIplus'] = 100*(df['DMplus_avg']/df['TR_avg'])
    df['DIminus'] = 100*(df['DMminus_avg']/df['TR_avg'])
    df['DIsum']=df['DIplus']+df['DIminus']
    df['DIdiff'] = abs(df['DIplus']-df['DIminus'])
    df['DX'] = 100*(df['DIdiff']/df['DIsum'])
    
    #ADX = ((n-1)*Previous ADX + Curr DX)/n
    
    for i in range(len(df)):
        if i<2*(n-1):
            ADXf.append(np.NaN)
        elif i==2*(n-1):
            ADXf.append(df['DX'][i-n+1:i+1].mean())
        elif i>2*(n-1):
            ADXf.append(((n-1)*ADXf[i-1] + df['DX'][i])/n)
    df['ADX']=np.array(ADXf)
    return df['ADX']

### On Balance Volume(OBV)
####  DF=Open, High, Low, Close, Volume pandas dataframe

In [ ]:
def OBV(DF):
    df = DF.copy()
    df['direction']= np.where(df['Close']>=df['Close'].shift(1), 1, -1)
    df['direction'][0] = 0
    df['V_direc']= df['direction']*df['Volume']
    df['obv'] = df['V_direc'].cumsum()
    return df['obv']